# Introduction

The goal of this project is to develop a library for performing automatic differentiation (AD), which we will call ```LazyDiff```.
AD is a set of techniques to numerically evaluate the derivative of a function motivated by deficiencies in classical methods. Numerical approximation is often not accurate enough and can be computationally expensive and symbolic differentiation tends to lead to inefficient code and faces the difficulty of converting a computer program into a single expression. Both classical methods have problems with calculating higher derivatives, where the complexity and errors increase, and are slow when it comes to computing the partial derivatives of a function with respect to many inputs. Automatic differentiation endeavors to solve all of these problems, and has been used in neural networks for back propagation and weight adjustment based on the loss function, and in scientific computing where it is often difficult to analytically compute the derivatives.





# Background
 AD exploits the fact that every function, no matter how complicated, is a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (such as $\sin$, $\cos$, $\exp$, $\log$, etc.). By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to working precision, and using at most a small constant factor more arithmetic operations than the original program.
 * **Chain Rule**  \\
   - AD relies heavily on the chain rule from calculus, which says that $$\frac{d}{dx} f(g(x)) = \frac{df}{dg} \frac{dg}{dx} $$ For a more complex composition $$y = f(g(h(x))) = f(g(h(x_1))) = f(g(x_2)) = f(x_3) = x_4$$ the chain rule gives   \\
    $$\frac{dy}{dx} = \frac{dy}{dx_3} \frac{dx_3}{dx_2} \frac{dx_2}{dx_1}\frac{dx_1}{dx}$$  \\
    
 * **Graph structure of calculations**:  \\
 We can think of taking derivatives in AD as building a graph representing evaluation of a function, and using the chain rule to propagate derivative values from input variables to the final function, where each node represents an elementary operation or elementary function.
 
 Below is a sample such graph structure resulting from evaluating the function $$f(x_1,x_2) = \sin(x_1)  + x_1 x_2$$
 
 ![alt text](ForwardAccumulationAutomaticDifferentiation.png)
 [image source](https://upload.wikimedia.org/wikipedia/commons/a/a4/ForwardAccumulationAutomaticDifferentiation.png)
 
 
 We see that the input values $x_1,x_2$ and their derivatives $\dot{w_1}, \dot{w_2}$ propagate upward and are used in computations for the derivatives $\dot{w_3}, \dot{w_4}, \dot{w_5}$ at each node on the way up to the final function $f(x_1,x_2) = w_5$. 
 
<!--  ![alt text](https://upload.wikimedia.org/wikipedia/commons/a/a0/ReverseaccumulationAD.png =400x200)   \\ -->
 

  
 


# How to Use LazyDiff

The user should interact with the package by creating functions or operations provided from the package. Once functions have been created, they can freely manipulate them by using elementary operations like +, -, * which will be overloaded as operations from the package. However, at first these fuctions will not take any values and combining functions will only link them together. These functions will be considered as the AD objects used for autodifferentiation.

To compute values, users will need to create variables using Var() function. These variables are needed as input to these functions to generate the corresponding outputs. See the code example below for an illustration.

To use the package, users will need to import the operations as ops and the variables class.

In [0]:
from lazydiff import ops, variables as var

def f(x, y):
  return ops.pow(ops.sin(x), 2) + y + 3

x = var.Var(3)
y = var.Var(4)
# returns a scalar value
result = f(x, y)
# returns tuple with containing gradient of f evaluated at x, y
gradient = result.grad(x, y)

# Software Organization

We will have a directory structure that resembles the following:
* lazydiff
    * lazydiff
        * `__init__.py`
        * `variables.py`
        * `ops.py`
        * tests
            * `basic_var_tests.py`
            * `ops_tests.py`
        * extensions
            * `<relevant_classes>.py`
    * README.md
    * LICENSE
    * setup.py
    * requirements.txt
    * .travis.yml
    * setup.cfg

The key modules included will be `variables.py`, which will contain our base classes that represent variables and functions for which we may want to compute, and `ops.py`, which will contain methods that allow us to evaluate elementary functions such as $\sin$, $\cos$, etc.

Additional extensions, such as reverse mode, backpropagation, etc. (TBD at this point), will be included in the `extensions` folder. 

The test suite will live in the folder "tests" in the directory structure above, and we will use Travic CI for continuous integration and Coveralls for test coverage.

Finally, we will distribute our packaged library through PyPI.

# Implementation

Our implementation will rely on numpy for evaluating elementary functions.

* In module `variables` :
    * We will have a class `Var` which contains:
        * Constructor which takes in a value `val`
            * Initializes empty gradient dictionary `grad_val`, which will contain derivatives of this variable with respect to other variables.
            * Initializes list representing gradient calculation parents (tuples of parent variables and weight of the gradients of each parent). For instance, if we are evaluating the derivative of $\sin x$ at $x = 3$, $$\frac{d}{dx} \sin x \Big|_{x = 3}= \dot{x}|_{x=3} \cos 3$$ so the list would include the tuple $(x, \cos 3)$.
        * Method ```grad``` which takes in variables, and returns a tuple containing the derivative with respect to those variables. 
        * Methods `__add__`, `__mul__` etc for overloading basic arithmetic operations
* In module `ops`:
    * We will have a method for each elementary function (such as `sin`, `cos`, `exp`, etc.). Each method will:
        * Take in a `Var` object
        * Create a new `Var` object with value updated to reflect execution of elementary function on value of input `Var` object, updated gradient calculation parent list
    